In [1]:
import pandas as pd
import numpy as np
import glob
 
import os
import zipfile

import tensorflow as tf

from keras_preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.models import load_model

In [2]:
# 다운로드 받은 이미지 압축 파일 해제
# print(os.listdir("./drive/MyDrive"))
local_zip = './train.zip'

zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('./res/train')
zip_ref.close()

local_zip = './test.zip'

zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('./res/test')
zip_ref.close()

In [3]:
base_dir = "./res/"

train_dir = "train/train/"

train_class = ['dog', 'elephant', 'giraffe', 'guitar', 'horse', 'house', 'person']


test_dir = "test/test/0"

In [4]:
train_one_hot = []

for path in range(len(train_class)):
    class2label = [''] + [0] * len(train_class)
    for img in os.listdir(base_dir + train_dir + train_class[path]):
        class2label[0] = (train_dir + train_class[path] + "/" + img)[6:]
        class2label[path + 1] = 1
        train_one_hot.append(class2label)

        
train_answer = pd.DataFrame(train_one_hot, columns = ["path", "dogs", 'elephant', 'giraffe', 'guitar', 'horse' , 'house', 'person'])
train_answer.to_csv('./train_answer.csv', index=False)

data = pd.read_csv("./train_answer.csv")
columns = data.columns

datagen=ImageDataGenerator(
      rescale = 1/255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
train_generator=datagen.flow_from_dataframe(
                                            dataframe=data,
                                            directory='./res/train/',
                                            x_col="path",
                                            y_col=columns[1:],
                                            batch_size=48,
                                            shuffle=False,
                                            class_mode="raw",
                                            target_size=(299,299))

Found 1698 validated image filenames.


In [13]:
res_net = tf.keras.applications.Xception(
    include_top=True, weights='imagenet',input_shape=(299,299,3), pooling="max")
res_net.trainable = True

pretrained_data = res_net.predict(train_generator, verbose=1)

36/36 [==============================] - 30s 806ms/step


In [15]:
x_train, x_valid, y_train, y_valid = train_test_split(pretrained_data, data.iloc[:,1:], test_size=0.2, random_state=42)

array([[7.2097086e-05, 5.3558592e-04, 8.1427446e-05, ..., 6.6124747e-05,
        1.9878868e-04, 6.7231647e-04],
       [1.5801309e-04, 3.5659948e-04, 4.8437162e-04, ..., 1.1016689e-04,
        5.3969480e-04, 3.0196656e-04],
       [1.5867651e-04, 3.4133715e-04, 3.9816851e-04, ..., 8.8227040e-05,
        4.8274104e-04, 2.3053346e-04],
       ...,
       [1.4409344e-04, 1.4884046e-03, 3.6039634e-03, ..., 1.0978100e-04,
        1.1550710e-04, 4.8191348e-04],
       [1.2266262e-04, 5.0407468e-04, 1.4371080e-04, ..., 2.2496170e-05,
        1.7531107e-04, 4.3735665e-04],
       [1.4851277e-04, 1.3638451e-04, 1.0188392e-04, ..., 1.1943936e-04,
        1.7577046e-04, 3.7337904e-05]], dtype=float32)

In [7]:
model = Sequential(
    [Input(1000,),
    BatchNormalization(),
    
    Dense(512,  activation='relu'),
    BatchNormalization(),
    
    Dense(256,  activation='relu'),
    BatchNormalization(),
    
    Dense(128,  activation='relu'),
    BatchNormalization(),
    
    Dense(7,  activation='softmax')
    ])
model.summary()
 
mc = tf.keras.callbacks.ModelCheckpoint('xception_model.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_4 (Batch (None, 1000)              4000      
_________________________________________________________________
dense (Dense)                (None, 512)               512512    
_________________________________________________________________
batch_normalization_5 (Batch (None, 512)               2048      
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
batch_normalization_6 (Batch (None, 256)               1024      
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
_________________________________________________________________
batch_normalization_7 (Batch (None, 128)               5

In [8]:
model.compile(loss='categorical_crossentropy', optimizer=Adam(0.001), metrics=['accuracy']) # 최적화 함수 학습률 1e-4에서 0.001로 변경
model.fit(x_train, y_train, epochs=30, batch_size=48, validation_data=(x_valid, y_valid),callbacks=[mc])

Epoch 1/30
29/29 [==============================] - 2s 25ms/step - loss: 0.1672 - accuracy: 0.9514 - val_loss: 1.8452 - val_accuracy: 0.4088

Epoch 00001: val_loss improved from inf to 1.84517, saving model to xception_model.h5
Epoch 2/30
29/29 [==============================] - 1s 18ms/step - loss: 0.0038 - accuracy: 1.0000 - val_loss: 1.8372 - val_accuracy: 0.2765

Epoch 00002: val_loss improved from 1.84517 to 1.83718, saving model to xception_model.h5
Epoch 3/30
29/29 [==============================] - 1s 18ms/step - loss: 0.0044 - accuracy: 0.9985 - val_loss: 1.8380 - val_accuracy: 0.2618

Epoch 00003: val_loss did not improve from 1.83718
Epoch 4/30
29/29 [==============================] - 1s 18ms/step - loss: 0.0017 - accuracy: 1.0000 - val_loss: 1.8256 - val_accuracy: 0.2882

Epoch 00004: val_loss improved from 1.83718 to 1.82564, saving model to xception_model.h5
Epoch 5/30
29/29 [==============================] - 1s 18ms/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 1.79

In [9]:
loaded_model = load_model('xception_model.h5')
 
final_model = Sequential([res_net, model])
final_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Functional)        (None, 1000)              22910480  
_________________________________________________________________
sequential (Sequential)      (None, 7)                 685223    
Total params: 23,595,703
Trainable params: 23,537,383
Non-trainable params: 58,320
_________________________________________________________________


In [10]:
test_df = pd.DataFrame()
test = glob.glob("./res/test/test/0/*.jpg")
test_dir = []
for img in test:
    test_dir.append(img[11:])
test_df["path"] = test_dir
 
test_datagen=ImageDataGenerator(rescale = 1/255)
test_generator = test_datagen.flow_from_dataframe(  dataframe=test_df[:],
                                                    directory='./res/test/',
                                                    x_col="path",
                                                    y_col=columns[0],
                                                    batch_size=48,
                                                    shuffle=False,
                                                    class_mode="raw",
                                                    target_size=(299,299))

Found 350 validated image filenames.


In [11]:
pred = final_model.predict(test_generator, verbose=1)

8/8 [==============================] - 36s 5s/step


In [12]:
answer = np.array([y.argmax() for y in pred])
test_df = pd.read_csv("./test_answer_sample_.csv")
test_df.iloc[:,1] = answer
test_df.to_csv('xception.csv', index=False)